# 🎯 Clean Google Maps Review Scraper

**Completely redesigned approach focusing on individual reviewers**

This scraper properly extracts:
- 📝 **Reviewer Name**: Individual person who wrote the review
- ⭐ **Star Rating**: Stars given by that specific reviewer
- 💬 **Review Text**: The actual review content
- 🏢 **Business/Apartment Name**: Location being reviewed

In [33]:
# 📚 Import Required Libraries
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

print("📚 All libraries imported successfully!")
print("🎯 Ready to scrape Google Maps reviews with precision!")

📚 All libraries imported successfully!
🎯 Ready to scrape Google Maps reviews with precision!


In [34]:
def setup_driver(headless=False):
    """
    Setup Chrome driver with optimal settings for Google Maps
    """
    try:
        options = Options()
        
        # Anti-detection settings
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        
        if headless:
            options.add_argument('--headless')
        
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.maximize_window()
        
        # Remove webdriver property
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        print("✅ Chrome driver setup complete")
        return driver
        
    except Exception as e:
        print(f"❌ Driver setup failed: {e}")
        return None

print("🔧 Driver setup function ready")

🔧 Driver setup function ready


In [35]:
def get_business_name(driver):
    """
    Extract the business/apartment name from Google Maps
    """
    name_selectors = [
        "h1[data-attrid='title']",
        "h1.DUwDvf",
        "h1.fontHeadlineLarge", 
        "div.lMbq3e h1",
        "h1"
    ]
    
    for selector in name_selectors:
        try:
            element = driver.find_element(By.CSS_SELECTOR, selector)
            name = element.text.strip()
            if name and len(name) > 2:
                print(f"🏢 Found business: {name}")
                return name
        except:
            continue
    
    return "Unknown Business"

def navigate_to_reviews(driver):
    """
    Navigate to the reviews tab in Google Maps
    """
    try:
        wait = WebDriverWait(driver, 15)
        
        # Look for Reviews button/tab
        review_selectors = [
            "button[data-value='Reviews']",
            "[data-tab-index='1']",
            "button[aria-label*='Reviews']",
            "div[role='tab'][aria-label*='Reviews']"
        ]
        
        for selector in review_selectors:
            try:
                reviews_tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
                reviews_tab.click()
                print("📝 Clicked on Reviews tab")
                time.sleep(3)
                return True
            except TimeoutException:
                continue
        
        print("ℹ️ Already on reviews or reviews visible")
        return True
        
    except Exception as e:
        print(f"⚠️ Navigation issue: {e}")
        return True  # Continue anyway

print("🏢 Business name and navigation functions ready")

🏢 Business name and navigation functions ready


In [36]:
def extract_reviewer_data(review_element):
    """
    Extract individual reviewer's name, stars, and review text
    IMPROVED: Better targeting of individual review components
    """
    try:
        reviewer_data = {
            'reviewer_name': 'Unknown',
            'stars': 'N/A', 
            'review_text': 'No text available'
        }
        
        # 1. Extract REVIEWER NAME - More specific targeting
        name_selectors = [
            ".d4r55",  # Primary reviewer name class
            "a .d4r55",  # Name within link
            ".TSUbDb .d4r55",
            ".fontHeaderMedium",
            "[aria-label] .d4r55"
        ]
        
        reviewer_name_found = False
        for selector in name_selectors:
            try:
                name_elements = review_element.find_elements(By.CSS_SELECTOR, selector)
                for name_elem in name_elements:
                    name = name_elem.text.strip()
                    if name and len(name) > 1 and len(name) < 100:  # Reasonable name length
                        reviewer_data['reviewer_name'] = name
                        reviewer_name_found = True
                        break
                if reviewer_name_found:
                    break
            except:
                continue
        
        # 2. Extract STAR RATING - Look for star images/aria-labels
        star_selectors = [
            "span[role='img'][aria-label*='star']",
            "div[role='img'][aria-label*='star']", 
            "[aria-label*='stars'][role='img']",
            "[aria-label*='Rated'][role='img']",
            ".kvMYJc[role='img']"
        ]
        
        stars_found = False
        for selector in star_selectors:
            try:
                star_elements = review_element.find_elements(By.CSS_SELECTOR, selector)
                for star_elem in star_elements:
                    aria_label = star_elem.get_attribute('aria-label')
                    if aria_label and ('star' in aria_label.lower() or 'rated' in aria_label.lower()):
                        # Extract rating from labels like "Rated 5 out of 5 stars" or "4 stars"
                        rating_match = re.search(r'(\d+(?:\.\d+)?)', aria_label)
                        if rating_match:
                            rating = rating_match.group(1)
                            if 1 <= float(rating) <= 5:  # Valid star rating
                                reviewer_data['stars'] = rating
                                stars_found = True
                                break
                if stars_found:
                    break
            except:
                continue
        
        # 3. Extract REVIEW TEXT - Look for actual review content
        text_selectors = [
            ".wiI7pd",  # Main review text
            ".MyEned .wiI7pd",
            "span.wiI7pd",
            "[data-expandable-section] .wiI7pd",
            ".review-text"
        ]
        
        review_text_found = False
        for selector in text_selectors:
            try:
                text_elements = review_element.find_elements(By.CSS_SELECTOR, selector)
                for text_elem in text_elements:
                    text = text_elem.text.strip()
                    # More strict filtering for actual review content
                    if (text and len(text) > 20 and len(text) < 2000 and  # Reasonable length
                        text != reviewer_data['reviewer_name'] and  # Not the name
                        'star' not in text.lower() and  # Not rating text
                        'rated' not in text.lower() and  # Not rating text
                        not text.isdigit() and  # Not just numbers
                        not re.match(r'^[\d\s\.,]+$', text)):  # Not just numbers/dates
                        reviewer_data['review_text'] = text[:500]  # Limit length
                        review_text_found = True
                        break
                if review_text_found:
                    break
            except:
                continue
        
        # Only return data if we found at least name and either stars or text
        if (reviewer_data['reviewer_name'] != 'Unknown' and 
            (reviewer_data['stars'] != 'N/A' or reviewer_data['review_text'] != 'No text available')):
            return reviewer_data
        else:
            return None
        
    except Exception as e:
        print(f"⚠️ Error extracting reviewer data: {e}")
        return None

print("👤 Individual reviewer extraction function ready")

👤 Individual reviewer extraction function ready


In [37]:
def scrape_reviews(driver, max_reviews):
    """
    Main function to scrape individual reviews from Google Maps
    Focus on getting exactly what we need: name, stars, review text
    FIXED: Now properly extracts unique individual reviewers
    """
    reviews = []
    seen_reviewers = set()  # Track unique reviewers to avoid duplicates
    
    print(f"🎯 Starting to scrape {max_reviews} UNIQUE individual reviews...")
    
    # Wait for reviews to load
    time.sleep(5)
    
    # Scroll down to load more reviews if needed
    for scroll_attempt in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    # Find review containers - try most specific first
    container_selectors = [
        "div[data-review-id]",  # Most reliable - actual review containers
        "div.jftiEf",  # Individual review blocks
        "div.MyEned",  # Alternative review container
    ]
    
    review_containers = []
    for selector in container_selectors:
        try:
            containers = driver.find_elements(By.CSS_SELECTOR, selector)
            if containers and len(containers) > 1:  # Need multiple containers for multiple reviews
                review_containers = containers
                print(f"📦 Found {len(containers)} review containers using: {selector}")
                break
        except:
            continue
    
    if not review_containers:
        print("❌ No review containers found")
        return []
    
    # Extract data from each unique review container
    count = 0
    processed_containers = 0
    
    for container in review_containers:
        if count >= max_reviews:
            break
            
        processed_containers += 1
        print(f"\n🔍 Processing container {processed_containers}/{len(review_containers)}")
            
        try:
            # Scroll container into view for better extraction
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", container)
            time.sleep(2)
            
            # Extract reviewer data
            reviewer_data = extract_reviewer_data(container)
            
            if reviewer_data and reviewer_data['reviewer_name'] != 'Unknown':
                reviewer_name = reviewer_data['reviewer_name']
                
                # Check if we've already seen this reviewer (avoid duplicates)
                if reviewer_name not in seen_reviewers:
                    seen_reviewers.add(reviewer_name)
                    reviews.append(reviewer_data)
                    count += 1
                    print(f"✅ UNIQUE Review {count}: {reviewer_name} | {reviewer_data['stars']}⭐")
                    print(f"   📝 Text: {reviewer_data['review_text'][:80]}...")
                else:
                    print(f"⚠️ Skipping duplicate reviewer: {reviewer_name}")
            else:
                print("⚠️ Could not extract valid reviewer data from this container")
            
        except Exception as e:
            print(f"❌ Error processing container: {e}")
            continue
    
    print(f"\n🎉 Successfully extracted {len(reviews)} UNIQUE reviews from {processed_containers} containers")
    return reviews

print("🔍 Main scraping function ready")

🔍 Main scraping function ready


In [38]:
def save_to_csv(reviews, business_name):
    """
    Save the extracted reviews to a CSV file
    """
    if not reviews:
        print("❌ No reviews to save")
        return False
    
    try:
        # Add business name to each review
        for review in reviews:
            review['business_name'] = business_name
        
        # Create DataFrame with proper column order
        df = pd.DataFrame(reviews)
        columns = ['business_name', 'reviewer_name', 'stars', 'review_text']
        df = df[columns]
        
        # Generate filename
        safe_name = business_name.replace(' ', '_').replace('/', '_')
        filename = f"clean_reviews_{safe_name}.csv"
        
        # Save to CSV
        df.to_csv(filename, index=False, encoding='utf-8')
        
        print(f"💾 Saved {len(reviews)} reviews to {filename}")
        print("\n📊 Sample data:")
        print(df.to_string(max_colwidth=60, index=False))
        
        return True
        
    except Exception as e:
        print(f"❌ Error saving CSV: {e}")
        return False

print("💾 CSV saving function ready")

💾 CSV saving function ready


In [39]:
def run_clean_scraper(url, max_reviews=2000, headless=False):
    """
    🚀 MAIN FUNCTION - Clean Google Maps Review Scraper
    
    This function orchestrates the entire scraping process:
    1. Sets up the browser
    2. Navigates to the URL  
    3. Gets the business name
    4. Navigates to reviews
    5. Extracts individual reviewer data
    6. Saves to CSV
    """
    driver = None
    
    try:
        print("🎯 CLEAN GOOGLE MAPS REVIEW SCRAPER")
        print("=" * 45)
        
        # Step 1: Setup driver
        print("🔧 Setting up Chrome driver...")
        driver = setup_driver(headless=headless)
        if not driver:
            return False
        
        # Step 2: Navigate to URL
        print(f"🌐 Loading: {url}")
        driver.get(url)
        time.sleep(5)
        
        # Step 3: Get business name
        business_name = get_business_name(driver)
        
        # Step 4: Navigate to reviews
        print("📝 Navigating to reviews section...")
        navigate_to_reviews(driver)
        
        # Step 5: Extract reviews
        reviews = scrape_reviews(driver, max_reviews=max_reviews)
        
        if reviews:
            # Step 6: Save to CSV
            save_to_csv(reviews, business_name)
            
            print("\n🎉 SCRAPING COMPLETED SUCCESSFULLY!")
            print(f"🏢 Business: {business_name}")
            print(f"📊 Reviews extracted: {len(reviews)}")
            return True
        else:
            print("\n❌ No reviews were extracted")
            return False
            
    except Exception as e:
        print(f"\n❌ Scraping failed: {e}")
        return False
        
    finally:
        if driver:
            driver.quit()
            print("\n✅ Browser closed")

print("🚀 Main scraper function ready to use!")

🚀 Main scraper function ready to use!


In [40]:
# 🎯 RUN THE CLEAN SCRAPER
# Replace the URL with your target Google Maps location

# Configuration
TARGET_URL = "https://maps.app.goo.gl/wT5sboiYx8TDCUgX7"  # Replace with your URL
MAX_REVIEWS = 2000  # Start with 2 reviews for testing
HEADLESS = False  # Set to True to hide browser

print("🎯 EXECUTING CLEAN SCRAPER - FIXED VERSION")
print("🔧 Now properly extracts UNIQUE individual reviewers")
print(f"📍 URL: {TARGET_URL}")
print(f"📊 Max Reviews: {MAX_REVIEWS}")
print("-" * 45)

# Run the scraper
success = run_clean_scraper(
    url=TARGET_URL,
    max_reviews=MAX_REVIEWS, 
    headless=HEADLESS
)

if success:
    print("\n✅ All done! Check your CSV file for UNIQUE reviewer results.")
else:
    print("\n❌ Scraping unsuccessful. Check the URL and try again.")

🎯 EXECUTING CLEAN SCRAPER - FIXED VERSION
🔧 Now properly extracts UNIQUE individual reviewers
📍 URL: https://maps.app.goo.gl/wT5sboiYx8TDCUgX7
📊 Max Reviews: 2000
---------------------------------------------
🎯 CLEAN GOOGLE MAPS REVIEW SCRAPER
🔧 Setting up Chrome driver...
✅ Chrome driver setup complete
🌐 Loading: https://maps.app.goo.gl/wT5sboiYx8TDCUgX7
🏢 Found business: The Holly
📝 Navigating to reviews section...
📝 Clicked on Reviews tab
🎯 Starting to scrape 2000 UNIQUE individual reviews...
📦 Found 20 review containers using: div[data-review-id]

🔍 Processing container 1/20
✅ UNIQUE Review 1: Bibek Dhungana | 1⭐
   📝 Text: I’m the guarantor for Apartment 627A. We have signed leases for both Summer and ...

🔍 Processing container 2/20
⚠️ Skipping duplicate reviewer: Bibek Dhungana

🔍 Processing container 3/20
✅ UNIQUE Review 2: Joshue Lewis | 5⭐
   📝 Text: I absolutely love living at The Holly! The location is unbeatable, especially fo...

🔍 Processing container 4/20
⚠️ Skipping du